# Домашнее задание №4

Это домашнее задание можно выполнять целиком в этом ноутбуке, либо алгоритмы написать в отдельном файле и импортировать сюда, для использования. В папке data лежат два файла islands.fasta и nonIslands.fasta. В них хранятся прочтения из CpG островков и из обычных участков генома соответственно, этими данными нужно будет воспользоваться в первом задании.

## Задача №1 (1)
Определите частоты генерации для каждого из нуклеотидов внутри CpG островков и вне их. Посчитайте так-же частоты для всех упорядоченных пар нуклеотидов и сравните частоту пары CG внутри островков и снаружи. Сделайте вывод. 

In [65]:
import pandas as pd
import itertools
from collections import defaultdict
from typing import List, Dict, Tuple
import numpy as np

In [66]:
# support function

def calculate_frequencies(file_path: str):
    with open(file_path, "r") as f:
        sequences = f.read().splitlines()[1::2]
        frequencies = {"A": 0, "C": 0, "G": 0, "T": 0}
        pair_frequencies = {"".join(pair): 0 for pair in itertools.product(frequencies.keys(), repeat=2)}
        for sequence in sequences:
            for i, nucleotide in enumerate(sequence):
                frequencies[nucleotide] += 1
                if i > 0:
                    pair_frequencies[sequence[i-1:i+1]] += 1
        return frequencies, pair_frequencies

def print_matrix(matrix: List[List], title: str):
    df = pd.DataFrame(matrix, columns=["A", "C", "G", "T"], index=["A", "C", "G", "T"])
    print(title)
    print(df)


In [5]:
# Calculate frequencies and pair frequencies

islands_freq, islands_pair_freq = calculate_frequencies("islands.fasta")
non_islands_freq, non_islands_pair_freq = calculate_frequencies("nonIslands.fasta")

In [6]:
print("Frequencies inside CpG islands:")
for nucleotide, frequency in islands_freq.items():
    print(f"{nucleotide}: {round(frequency / sum(islands_freq.values()), 6)}")
print()
print("Frequencies outside CpG islands:")
for nucleotide, frequency in non_islands_freq.items():
    print(f"{nucleotide}: {round(frequency / sum(non_islands_freq.values()), 6)}")

Frequencies inside CpG islands:
A: 0.202462
C: 0.296397
G: 0.298277
T: 0.202864

Frequencies outside CpG islands:
A: 0.290077
C: 0.195946
G: 0.198234
T: 0.315744


In [7]:
# Calculate pair frequency matrix for CpG islands

cpg_islands_pair_freq = [[islands_pair_freq.get(n1 + n2, 0) / sum([islands_pair_freq.get(n1 + n3, 0) for n3 in "ACGT"]) for n2 in "ACGT"] for n1 in "ACGT"]
print_matrix(cpg_islands_pair_freq, "Pair frequency matrix for CpG islands:")

Pair frequency matrix for CpG islands:
          A         C         G         T
A  0.252965  0.226691  0.355276  0.165069
C  0.212177  0.346783  0.198988  0.242052
G  0.200725  0.295684  0.347172  0.156420
T  0.140276  0.293477  0.314688  0.251559


In [8]:
# Calculate pair frequency matrix for non-CpG islands

non_cpg_islands_pair_freq = [[non_islands_pair_freq.get(n1 + n2, 0) / sum([non_islands_pair_freq.get(n1 + n3, 0) for n3 in "ACGT"]) for n2 in "ACGT"] for n1 in "ACGT"]
print_matrix(non_cpg_islands_pair_freq, "Pair frequency matrix for non-CpG islands:")

Pair frequency matrix for non-CpG islands:
          A         C         G         T
A  0.314032  0.160684  0.240730  0.284553
C  0.343869  0.253536  0.027851  0.374744
G  0.308569  0.172387  0.251412  0.267632
T  0.222417  0.207934  0.231397  0.338252


In [9]:
# Calculate CG frequency inside and outside CpG islands

cpg_islands_cg_freq = islands_pair_freq.get("CG", 0) / sum([islands_pair_freq.get("C" + n, 0) for n in "ACGT"])
non_cpg_islands_cg_freq = non_islands_pair_freq.get("CG", 0) / sum([non_islands_pair_freq.get("C" + n, 0) for n in "ACGT"])
print()
print(f"CG frequency inside CpG islands: {round(cpg_islands_cg_freq, 5)}")
print()
print(f"CG frequency outside CpG islands: {round(non_cpg_islands_cg_freq, 5)}")


CG frequency inside CpG islands: 0.19899

CG frequency outside CpG islands: 0.02785


Таким образом, вероятность появления CpG выше внутри островков, чем снаружи

## Задача №2 (2)
Напишите марковскую модель, которая имеет открытые состояния {A, T, G, C}, и скрытые состояния {+, -}. Когда модель в состоянии **+**, то вероятность генерации некоторого символа нуклеотида соответствует его частоте внутри CpG островков, вычислиному в первом задании, если состояние **-**, то частоте вне островков. Вероятность остаться внутри островка 0.95, а перейти в обычный геном 0.05. Для остальной части генома соответствующие вероятности 0.995 и 0.005. Саму модель можно реализовать в виде итератора, определив метод next, который возвращает пару - состояние и нуклеотид, который в этом состоянии произведен.    
Воспользуйтесь данной моделью для того чтобы сгенерировать набор из 20 последовательностей длинной от 1 000 до 100 000, причем к каждой последовательности должна прилагаться последовательность состояний.

In [10]:
# Calculate probabilities of nucleotides inside and outside CpG islands

islands_prob = {nucleotide: islands_freq[nucleotide] / sum(islands_freq.values()) for nucleotide in "ACGT"}
non_islands_prob = {nucleotide: non_islands_freq[nucleotide] / sum(non_islands_freq.values()) for nucleotide in "ACGT"}


In [11]:
# Define transition probabilities

island_transition_prob = 0.95
non_island_transition_prob = 0.005


In [67]:
import random

class MarkovModel:
    def __init__(self, islands_prob: Dict[str, float], non_islands_prob: Dict[str, float], island_transition_prob: float, non_island_transition_prob: float):
        self.islands_prob = islands_prob
        self.non_islands_prob = non_islands_prob
        self.island_transition_prob = island_transition_prob
        self.non_island_transition_prob = non_island_transition_prob
        self.current_state = "+"
        
    def next(self):
        if self.current_state == "+":
            nucleotide = random.choices(list(self.islands_prob.keys()), weights=list(self.islands_prob.values()))[0]
            if random.random() < self.island_transition_prob:
                self.current_state = "+"
            else:
                self.current_state = "-"
        else:
            nucleotide = random.choices(list(self.non_islands_prob.keys()), weights=list(self.non_islands_prob.values()))[0]
            if random.random() < self.non_island_transition_prob:
                self.current_state = "-"
            else:
                self.current_state = "+"
        
        return (self.current_state, nucleotide)


In [78]:
for i in range(20):
    length = random.randint(100, 10000)
    model = MarkovModel(islands_prob, non_islands_prob, island_transition_prob, non_island_transition_prob)
    sequence = ""
    states = ""
    for j in range(length):
        state, nucleotide = model.next()
        states += state
        sequence += nucleotide


## Задача №3 (4)
Напишите алгоритм Витерби для восстановления последовательности скрытых состояний марковской модели из второго задаания. Воспользуйтесь им, воссстановив состояния тех последовательностей, которые вы получили во втором задании и посчитайте TP, TN, FP, FN по количеству правильно или ошибочно предсказанных позиций из CpG остравков. 

In [73]:
import numpy as np

def viterbi_algorithm(states: str, sequence: str, islands_prob: Dict[str, float], non_islands_prob: Dict[str, float], p_islands: float = 0.95, p_non_islands: float = 0.005):
    # Создаем матрицу вероятностей
    prob_matrix = np.zeros((len(states), len(sequence)))
    # Заполняем первый столбец матрицы вероятностей
    prob_matrix[:, 0] = [islands_prob[sequence[0]] if s == '+' else non_islands_prob[sequence[0]] for s in states]
    # Создаем матрицу для хранения предыдущих состояний
    prev_states = np.zeros((len(states), len(sequence)), dtype=int)
    # Заполняем матрицы вероятностей и предыдущих состояний
    for j in range(1, len(sequence)):
        p_emit = np.array([islands_prob[sequence[j]] if s == '+' else non_islands_prob[sequence[j]] for s in states])
        p_prev = prob_matrix[:, j-1] * (p_islands if states == '+' else p_non_islands)
        prob_matrix[:, j] = p_emit * np.max(p_prev, axis=0)
        prev_states[:, j] = np.argmax(p_prev, axis=0)
    # Выбираем последнее состояние с максимальной вероятностью
    last_state = np.argmax(prob_matrix[:, -1])
    # Строим последовательность скрытых состояний с помощью матрицы предыдущих состояний
    hidden_states = [last_state]
    for j in range(len(sequence)-1, 0, -1):
        last_state = prev_states[last_state, j]
        hidden_states.insert(0, last_state)
    return ''.join(states[i] for i in hidden_states)


pred_states = viterbi_algorithm(states, sequence, islands_prob, non_islands_prob)

In [75]:
TP = sum(1 for i in range(len(states)) if states[i] == '+' and pred_states[i] == '+')
TN = sum(1 for i in range(len(states)) if states[i] == '-' and pred_states[i] == '-')
FP = sum(1 for i in range(len(states)) if states[i] == '-' and pred_states[i] == '+')
FN = sum(1 for i in range(len(states)) if states[i] == '+' and pred_states[i] == '-')


def accuracy(TP: int, TN: int, FP: int, FN: int):
    right = TP + TN
    wrong = FP + FN
    total = right + wrong
    return right / total

print(f"Result: ") 
print(f"correct={round(accuracy(TP, TN, FP, FN), 4)}, ")
print(f"TP={TP}, TN={TN}, FP={FP}, FN={FN}")

Result: 
correct=0.9496, 
TP=16683, TN=4, FP=840, FN=45


## Задача №4 (4)
Напишите алгоритм вперед назад для модели из второго задания. Пользуясь этим алгоритмом найдите вероятности того, что модель находилась в состоянии **+** для каждой позиции строк из второго задания. Устанавливая различные пороговые значения, определите позиции соответствующие CpG островкам и посчитайте TP. Постройте график зависимости TP от выбранного порогового значения. Есть ли пороговые значения при которых TP больше чем в задании №3?